In [7]:
# Read in
import pandas as pd
import numpy as np

# bike_file_path = "../data/Bike-Sharing-Dataset/hour.csv"
# loan_file_path = "../data/loan-default-data/Training Data.csv"

bike_file_path = "Project/Bike-Sharing-Dataset/hour.csv"
loan_file_path = "Project/loan-default-data/Training Data.csv"

def read_in(file_path):
    df = pd.read_csv(file_path)
    if file_path is bike_file_path:
        y = df["cnt"]
        X = df.drop(["cnt"], axis = 1)
    if file_path is loan_file_path:
        y = df["Risk_Flag"]
        X = df.drop(["Risk_Flag"], axis = 1)
    y = pd.DataFrame(y)
    return y, X

In [91]:
y, X = read_in(bike_file_path)
y = pd.DataFrame(y)


,cnt
0,16
1,40
2,32
3,13
4,1
...,...
17374,119
17375,89
17376,90
17377,61


In [99]:
fractionObserved = .9
Omega = np.array(np.random.rand(X.shape[0],X.shape[1])< fractionObserved)
Xobs = Omega*X
Xobs

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,0,1,0,0,0,0,1,0.24,0.2879,0.81,0.0000,3,13
1,0,0,1,0,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,0,27
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10
4,1,0,1,4,0,0,0,0,0.24,0.2879,0.75,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,1,12,19,0,1,1,0,0.26,0.2576,0.60,0.1642,11,108
17375,1,1,12,20,0,1,0,2,0.26,0.2576,0.60,0.1642,8,81
17376,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83
17377,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48


In [96]:
X = X.drop(["dteday", "instant"], axis=1)

In [11]:
def impute_mean(Omega, X):
    '''
    Take the data with missing values and impute with the column mean
    '''
    new = X*Omega
    means = new.mean(axis=0)
    X = X.mask(df*(1-Omega)==1).fillna(means)
    return X

In [87]:
def delete_missing(Omega, y, X):
    '''
    Drop the rows with missing values in both the X and y datasets
    '''
    final = []
    for i in [y,X]:
        r = Omega.shape[1] - Omega.sum(axis = 1)
        i["keeps"] = r
        new = i.loc[y['keeps'] == 0]
        new = new.drop(["keeps"], axis =1)
        final.append(new)
    return final[0], final[1]

In [104]:
def singular_value_thresholding(Omega, X):
    '''
    Use singular value thresholding to loop through and fill the missing values
    '''
    
    Xobs = Omega*X
    tau = 30
    stopping_value = 0.1
    X_hat = Xobs
    X_old = np.zeros((X.shape[0],X.shape[1]))

    while np.linalg.norm(X_hat -X_old) > stopping_value:
        X_old = X_hat
        u, s, vt = np.linalg.svd(X_hat, full_matrices=False)
        st = np.where(s > tau,s,0)
        X_new = u@np.diag(st)@vt
        X_hat = Xobs*Omega + X_new*(1-Omega)

    return X

In [105]:
new_X = singular_value_thresholding(Omega, X)
new_X

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108
17375,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81
17376,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83
17377,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48


In [107]:
Xobs

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,0,1,0,0,0,0,1,0.24,0.2879,0.81,0.0000,3,13
1,0,0,1,0,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,0,27
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10
4,1,0,1,4,0,0,0,0,0.24,0.2879,0.75,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,1,12,19,0,1,1,0,0.26,0.2576,0.60,0.1642,11,108
17375,1,1,12,20,0,1,0,2,0.26,0.2576,0.60,0.1642,8,81
17376,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83
17377,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48


In [ ]:
import random

seed = 4358

def cross_validation(X, y, seed):
    
    error_calc = []
    
    for i in range(30):
        random.seed(seed)

        Omega = missing_obs(seed, X)
        impute_mean_X = impute_mean(Omega, X)
        deletion_y, deletion_X = delete_missing(Omega, y, X)
        svt_X = singular_value_thresholding(Omega, X)
        KNN_X = KNN(Omega, X)
        EM_X = EM(Omega, X)

        

        seed = seed/i
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
